## Parse Extraction Data

In [1]:
import json
import re
from dataclasses import dataclass, field
from typing import Optional, TypedDict, Dict, List, Tuple, Set
from collections import defaultdict
from math import ceil

In [2]:
@dataclass
class ResourceAmount:
    resource: "Resource"
    amount: int


@dataclass
class ResourceEfficiency:
    resource: "Resource"
    factor: float


@dataclass
class ProcessingStep:
    input: "Resource"
    tool: str
    catalysts: list[ResourceEfficiency] = field(default_factory=list)
    outputs: list[ResourceEfficiency] = field(default_factory=list)


class Resource:
    def __init__(self, name: str):
        self.name: str = name
        self.value: float | None = None
        self.downstream: list[ProcessingStep] = []
        self.upstream: list[ProcessingStep] = []

    def __str__(self):
        return self.name

    def __repr__(self):
        return f"Resource({self.name})"

    def add_downstream(self, step: ProcessingStep) -> list[ProcessingStep]:
        self.downstream.append(step)
        return self.downstream

    def add_upstream(self, step: ProcessingStep) -> list[ProcessingStep]:
        self.upstream.append(step)
        return self.upstream


In [3]:
resources: dict[str, Resource] = {}


def get_resource_by_name(name: str) -> Resource:
    return resources.setdefault(name, Resource(name))

## Parse Extraction Data

In [4]:
# Original data taken from:
#  https://docs.google.com/spreadsheets/d/1CRb27W_1AjjxisAlK5KHTLXkhUjjQfKO7_hmh5eY7L4/edit#gid=0
#  (Preprocessed to normalize null values and fix typos)
#  with some supplemental data from https://mortal2.rocks/guides/ore-extraction-calculator/ (not fully imported)
NORSCA_SHEET_CLEANED = """
[
  {
    "Input": "Granum (10k)",
    "Catalyst": null,
    "Tool": "Crusher",
    "Output 1": "Amarantum (882)",
    "Output 2": "Blood Ore (770)",
    "Output 3": "Flakestone (140)",
    "Output 4": "Granum Powder (2940)",
    "Output 5": null
  },
  {
    "Input": "Granum (10k)",
    "Catalyst": "Calx Powder (715)",
    "Tool": "Attractor",
    "Output 1": null,
    "Output 2": "Blood Ore (1980)",
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Calx (10k)",
    "Catalyst": null,
    "Tool": "Crusher",
    "Output 1": "Calspar (360)",
    "Output 2": "Malachite (891)",
    "Output 3": "Flakestone (180)",
    "Output 4": "Coal (2151)",
    "Output 5": "Calx Powder (1361)"
  },
  {
    "Input": "Calx (10k)",
    "Catalyst": "Water (1000)",
    "Tool": "Grinder",
    "Output 1": "Calspar (2000)",
    "Output 2": "Malachite (528)",
    "Output 3": "Flakestone (36)",
    "Output 4": "Coal (1140)",
    "Output 5": "Calx Powder (2058)"
  },
  {
    "Input": "Calx (10k)",
    "Catalyst": "Water (1000)",
    "Tool": "Furnace",
    "Output 1": "Calspar (2560)",
    "Output 2": "Malachite (506)",
    "Output 3": "Flakestone (28)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Saburra (10k)",
    "Catalyst": null,
    "Tool": "Crusher",
    "Output 1": "Bleckblende (1584)",
    "Output 2": "Malachite (1584)",
    "Output 3": "Jadeite (128)",
    "Output 4": "Pyrite (32)",
    "Output 5": "Saburra Powder (2000)"
  },
  {
    "Input": "Saburra (10k)",
    "Catalyst": "Water (1000)",
    "Tool": "Grinder",
    "Output 1": "Bleckblende (1901)",
    "Output 2": "Malachite (950)",
    "Output 3": "Jadeite (16)",
    "Output 4": null,
    "Output 5": "Saburra Powder (4275)"
  },
  {
    "Input": "Saburra (10k)",
    "Catalyst": "Coal (535)",
    "Tool": "Attractor",
    "Output 1": null,
    "Output 2": null,
    "Output 3": null,
    "Output 4": "Pyrite (120)",
    "Output 5": null
  },
  {
    "Input": "Cerulite (10k)",
    "Catalyst": null,
    "Tool": "Crusher",
    "Output 1": "Azurite (2200)",
    "Output 2": "Malachite (5760)",
    "Output 3": null,
    "Output 4": "Pyrite (80)",
    "Output 5": "Suburra Powder (500)"
  },
  {
    "Input": "Ritualist (10k)",
    "Catalyst": null,
    "Tool": "Furnace",
    "Output 1": "Unholy Ash (105)",
    "Output 2": null,
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Gabore (10k)",
    "Catalyst": null,
    "Tool": "Crusher",
    "Output 1": "Galbinum (750)",
    "Output 2": "Blood Ore (510)",
    "Output 3": "Nyx (36)",
    "Output 4": null,
    "Output 5": "Gabore Powder (2340)"
  },
  {
    "Input": "Gabore (10k)",
    "Catalyst": "Bor (720)",
    "Tool": "Grinder",
    "Output 1": "Galbinum (1050)",
    "Output 2": "Blood Ore (226)",
    "Output 3": null,
    "Output 4": null,
    "Output 5": "Gabore Powder (4168)"
  },
  {
    "Input": "Gabore (10k)",
    "Catalyst": "Coke (385)",
    "Tool": "Furnace",
    "Output 1": "Galbinum (243)",
    "Output 2": "Blood Ore (595)",
    "Output 3": "Nyx (168)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Tephra (10k)",
    "Catalyst": null,
    "Tool": "Crusher",
    "Output 1": "Galbinum (1200)",
    "Output 2": "Cinnabar (144)",
    "Output 3": "Magmum (288)",
    "Output 4": "Red Bleckblende (312)",
    "Output 5": "Volcanic Ash (2220)"
  },
  {
    "Input": "Tephra (10k)",
    "Catalyst": "Bor (720)",
    "Tool": "Grinder",
    "Output 1": "Galbinum (1200)",
    "Output 2": "Cinnabar (90)",
    "Output 3": "Magmum (259)",
    "Output 4": "Red Bleckblende (807)",
    "Output 5": "Volcanic Ash (2753)"
  },
  {
    "Input": "Tephra (10k)",
    "Catalyst": "Water (1000)",
    "Tool": "Grinder",
    "Output 1": "Galbinum (1900)",
    "Output 2": "Cinnabar (88)",
    "Output 3": "Magmum (259)",
    "Output 4": "Red Bleckblende (774)",
    "Output 5": "Volcanic Ash (2960)"
  },
  {
    "Input": "Tephra (10k)",
    "Catalyst": "Calx Powder (715)",
    "Tool": "Greater Nat",
    "Output 1": "Galbinum (250)",
    "Output 2": "Cinnabar (36)",
    "Output 3": "Magmum (122)",
    "Output 4": "Red Bleckblende (2600)",
    "Output 5": "Volcanic Ash (148)"
  },
  {
    "Input": "Tephra (10k)",
    "Catalyst": "Bor (720)",
    "Tool": "Greater Nat",
    "Output 1": "Galbinum (1250)",
    "Output 2": "Cinnabar (94)",
    "Output 3": "Magmum (36)",
    "Output 4": "Red Bleckblende (1976)",
    "Output 5": "Volcanic Ash (326)"
  },
  {
    "Input": "Tephra (10k)",
    "Catalyst": "Calx Powder (715)",
    "Tool": "Furnace",
    "Output 1": "Galbinum (299)",
    "Output 2": "Cinnabar (54)",
    "Output 3": "Magmum (259)",
    "Output 4": "Red Bleckblende (2080)",
    "Output 5": "Volcanic Ash (444)"
  },
  {
    "Input": "Kimurite (10k)",
    "Catalyst": null,
    "Tool": "Crusher",
    "Output 1": "Waterstone (3800)",
    "Output 2": null,
    "Output 3": "Magmum (3520)",
    "Output 4": "Pyrite (200)",
    "Output 5": null
  },
  {
    "Input": "Blood Ore (10k)",
    "Catalyst": "Coke (385)",
    "Tool": "Furnace",
    "Output 1": "Pig Iron (4000)",
    "Output 2": null,
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Blood Ore (10k)",
    "Catalyst": "Coke (385)",
    "Tool": "Blast Furnace",
    "Output 1": "Pig Iron (5000)",
    "Output 2": null,
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Amarantum (10k)",
    "Catalyst": null,
    "Tool": "Crusher",
    "Output 1": "Waterstone (900)",
    "Output 2": "Electrum (180)",
    "Output 3": "Cuprum (270)",
    "Output 4": "Bleck (135)",
    "Output 5": "Calamine (90)"
  },
  {
    "Input": "Amarantum (10k)",
    "Catalyst": "Bor (720)",
    "Tool": "Furnace",
    "Output 1": null,
    "Output 2": "Electrum (480)",
    "Output 3": "Cuprum (2400)",
    "Output 4": "Bleck (960)",
    "Output 5": "Calamine (563)"
  },
  {
    "Input": "Amarantum (10k)",
    "Catalyst": "Bor (720)",
    "Tool": "Blast Furnace",
    "Output 1": null,
    "Output 2": "Electrum (800)",
    "Output 3": "Cuprum (3000)",
    "Output 4": "Bleck (600)",
    "Output 5": "Calamine (624)"
  },
  {
    "Input": "Calspar (10k)",
    "Catalyst": "Dragon Salt (190)",
    "Tool": "Furnace",
    "Output 1": "Chalk Glance (245)",
    "Output 2": "Electrum (88)",
    "Output 3": "Malachite (1445)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Calspar (10k)",
    "Catalyst": "Ichor (660)",
    "Tool": "Furnace",
    "Output 1": "Chalk Glance (159)",
    "Output 2": "Electrum (212)",
    "Output 3": "Malachite (2793)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Calspar (10k)",
    "Catalyst": "Sulfur (560)",
    "Tool": "Furnace",
    "Output 1": "Chalk Glance (142)",
    "Output 2": "Electrum (129)",
    "Output 3": "Malachite (1782)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Calspar (10k)",
    "Catalyst": "Water (1000)",
    "Tool": "Furnace",
    "Output 1": "Chalk Glance (72)",
    "Output 2": "Electrum (109)",
    "Output 3": "Malachite (3468)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Calspar (10k)",
    "Catalyst": "Bor (720)",
    "Tool": "Furnace",
    "Output 1": "Chalk Glance (72)",
    "Output 2": "Electrum (294)",
    "Output 3": "Malachite (4816)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Calspar (10k)",
    "Catalyst": "Coke (385)",
    "Tool": "Furnace",
    "Output 1": "Chalk Glance (72)",
    "Output 2": "Electrum (212)",
    "Output 3": "Malachite (3468)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Calspar (10k)",
    "Catalyst": "Dragon Salt (190)",
    "Tool": "Blast Furnace",
    "Output 1": "Chalk Glance (700)",
    "Output 2": "Electrum (224)",
    "Output 3": "Malachite (2064)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Calspar (10k)",
    "Catalyst": "Ichor (660)",
    "Tool": "Blast Furnace",
    "Output 1": "Chalk Glance (490)",
    "Output 2": "Electrum (426)",
    "Output 3": "Malachite (3302)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Calspar (10k)",
    "Catalyst": "Sulfur (560)",
    "Tool": "Blast Furnace",
    "Output 1": "Chalk Glance (291)",
    "Output 2": "Electrum (291)",
    "Output 3": "Malachite (2374)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Calspar (10k)",
    "Catalyst": "Dragon Salt (190)",
    "Tool": "Fabricula",
    "Output 1": "Chalk Glance (420)",
    "Output 2": null,
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Calspar (10k)",
    "Catalyst": "Ichor (660)",
    "Tool": "Fabricula",
    "Output 1": "Chalk Glance (210)",
    "Output 2": "Electrum (336)",
    "Output 3": "Malachite (1376)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Calspar (10k)",
    "Catalyst": "Sulfur (560)",
    "Tool": "Fabricula",
    "Output 1": "Chalk Glance (168)",
    "Output 2": "Electrum (112)",
    "Output 3": "Malachite (344)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Calspar (10k)",
    "Catalyst": "Bor (720)",
    "Tool": "Fabricula",
    "Output 1": null,
    "Output 2": "Electrum (560)",
    "Output 3": "Malachite (3440)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Malachite (10k)",
    "Catalyst": "Bor (720)",
    "Tool": "Furnace ",
    "Output 1": "Cuprum (4000)",
    "Output 2": "Sulfur (47)",
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Malachite (10k)",
    "Catalyst": "Coke (385)",
    "Tool": "Furnace ",
    "Output 1": "Cuprum (4000)",
    "Output 2": "Sulfur (83)",
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Malachite (10k)",
    "Catalyst": "Bor (720)",
    "Tool": "Blast Furnace",
    "Output 1": "Cuprum (5000)",
    "Output 2": null,
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Malachite (10k)",
    "Catalyst": "Bor (720)",
    "Tool": "Fabricula",
    "Output 1": "Cuprum (4000)",
    "Output 2": "Sulfur (64)",
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Malachite (10k)",
    "Catalyst": "Coke (385)",
    "Tool": "Fabricula",
    "Output 1": "Cuprum (4000)",
    "Output 2": "Sulfur (320)",
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Malachite (10k)",
    "Catalyst": "Coal (535)",
    "Tool": "Fabricula",
    "Output 1": "Cuprum (2400)",
    "Output 2": "Sulfur (640)",
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Bleckblende (10k)",
    "Catalyst": "Bor (720)",
    "Tool": "Natorus",
    "Output 1": "Bleck (5400)",
    "Output 2": null,
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Bleckblende (10k)",
    "Catalyst": "Rock Oil (420)",
    "Tool": "Natorus",
    "Output 1": "Bleck (5400)",
    "Output 2": null,
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Bleckblende (10k)",
    "Catalyst": "Bor (720)",
    "Tool": "Greater Nat",
    "Output 1": "Bleck (6000)",
    "Output 2": null,
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Bleckblende (10k)",
    "Catalyst": "Bor (720)",
    "Tool": "Furnace",
    "Output 1": "Bleck (1920)",
    "Output 2": null,
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Cinnabar (10k)",
    "Catalyst": "Bor (720)",
    "Tool": "Furnace",
    "Output 1": "Ichor (3900)",
    "Output 2": "Sulfur (296)",
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Cinnabar (10k)",
    "Catalyst": "Bor (720)",
    "Tool": "Greater Nat",
    "Output 1": null,
    "Output 2": "Sulfur (672)",
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Cinnabar (10k)",
    "Catalyst": "Bor (720)",
    "Tool": "Fabricula",
    "Output 1": "Ichor (960)",
    "Output 2": "Sulfur (160)",
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Coal (10k)",
    "Catalyst": "Coal (535)",
    "Tool": "Furnace",
    "Output 1": "Coke (7200)",
    "Output 2": "Pitch (418)",
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Coal (10k)",
    "Catalyst": "Calx Powder (715)",
    "Tool": "Furnace",
    "Output 1": "Coke (7200)",
    "Output 2": "Pitch (386)",
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Magmum (10k)",
    "Catalyst": null,
    "Tool": "Grizzly",
    "Output 1": "Kyanite (2500)",
    "Output 2": "Maalite (650)",
    "Output 3": "Pyropite (200)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Magmum (10k)",
    "Catalyst": "Bor (720)",
    "Tool": "Kiln",
    "Output 1": null,
    "Output 2": "Maalite (2080)",
    "Output 3": "Pyropite (480)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Magmum (10k)",
    "Catalyst": "Coke (385)",
    "Tool": "Kiln",
    "Output 1": null,
    "Output 2": "Maalite (4264)",
    "Output 3": "Pyropite (221)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Magmum (10k)",
    "Catalyst": "Bor (720)",
    "Tool": "Hearth",
    "Output 1": null,
    "Output 2": "Maalite (780)",
    "Output 3": "Pyropite (600)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Galbinum (10k)",
    "Catalyst": "Calx Powder (715)",
    "Tool": "Furnace",
    "Output 1": "Lupium (171)",
    "Output 2": "Pyrite (37)",
    "Output 3": "Pyroxene (696)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Galbinum (10k)",
    "Catalyst": "Calx Powder (715)",
    "Tool": "Blast Furnace",
    "Output 1": "Lupium (315)",
    "Output 2": null,
    "Output 3": "Pyroxene (1280)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Galbinum (10k)",
    "Catalyst": "Coke (385)",
    "Tool": "Blast Furnace",
    "Output 1": "Lupium (246)",
    "Output 2": null,
    "Output 3": "Pyroxene (1760)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Galbinum (10k)",
    "Catalyst": "Bor (720)",
    "Tool": "Blast Furnace",
    "Output 1": "Lupium (246)",
    "Output 2": null,
    "Output 3": "Pyroxene (1400)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Galbinum (10k)",
    "Catalyst": "Coke (385)",
    "Tool": "Greater Nat",
    "Output 1": "Lupium (62)",
    "Output 2": "Pyrite (653)",
    "Output 3": "Pyroxene (4200)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Azurite (10k)",
    "Catalyst": "Fuming Salt (315)",
    "Tool": "Fabricula",
    "Output 1": "Cuprite (300)",
    "Output 2": null,
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Electrum (10k)",
    "Catalyst": "Calx Powder (715)",
    "Tool": "Furnace ",
    "Output 1": "Cuprum (2600)",
    "Output 2": "Silver (1600)",
    "Output 3": "Gold (1400)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Electrum (10k)",
    "Catalyst": "Calx Powder (715)",
    "Tool": "Blast Furnace",
    "Output 1": "Cuprum (3500)",
    "Output 2": "Silver (2500)",
    "Output 3": "Gold (1750)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Chalk Glance (10k)",
    "Catalyst": "Dragon Salt (190)",
    "Tool": "Fabricula",
    "Output 1": "Skadite (3200)",
    "Output 2": null,
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Chalk Glance (10k)",
    "Catalyst": "Ichor (660)",
    "Tool": "Fabricula",
    "Output 1": "Skadite (3200)",
    "Output 2": null,
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Pyroxene (10k)",
    "Catalyst": "Calx Powder (715)",
    "Tool": "Blast Furnace",
    "Output 1": "Almine (400)",
    "Output 2": "Acronite (800)",
    "Output 3": "Electrum (2720)",
    "Output 4": "Calamine (2040)",
    "Output 5": null
  },
  {
    "Input": "Pyroxene (10k)",
    "Catalyst": "Calx Powder (715)",
    "Tool": "Greater Nat",
    "Output 1": "Almine (2000)",
    "Output 2": "Acronite (160)",
    "Output 3": "Electrum (2720)",
    "Output 4": "Calamine (3400)",
    "Output 5": null
  },
  {
    "Input": "Red Bleckblende (10k)",
    "Catalyst": "Coke (385)",
    "Tool": "Furnace",
    "Output 1": "Aabam (2322)",
    "Output 2": "Silver (198)",
    "Output 3": "Sanguinite (23)",
    "Output 4": "Calamine (569)",
    "Output 5": null
  },
  {
    "Input": "Red Bleckblende (10k)",
    "Catalyst": "Rock Oil (420)",
    "Tool": "Blast Furnace",
    "Output 1": "Aabam (3000)",
    "Output 2": "Silver (200)",
    "Output 3": "Sanguinite (140)",
    "Output 4": "Calamine (240)",
    "Output 5": null
  },
  {
    "Input": "Red Bleckblende (10k)",
    "Catalyst": "Fuming Salt (315)",
    "Tool": "Blast Furnace",
    "Output 1": "Aabam (3000)",
    "Output 2": "Silver (350)",
    "Output 3": "Sanguinite (200)",
    "Output 4": "Calamine (456)",
    "Output 5": null
  },
  {
    "Input": "Waterstone (10k)",
    "Catalyst": "Rock Oil (420)",
    "Tool": "Fabricula",
    "Output 1": "Gem Metal (3200)",
    "Output 2": null,
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Waterstone (10k)",
    "Catalyst": "Bor (720)",
    "Tool": "Fabricula",
    "Output 1": "Gem Metal (1920)",
    "Output 2": "Lupium (960)",
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Waterstone (10k)",
    "Catalyst": "Fuming Salt (315)",
    "Tool": "Fabricula",
    "Output 1": "Gem Metal (640)",
    "Output 2": "Lupium (2400)",
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Waterstone (10k)",
    "Catalyst": "Fuming Salt (315)",
    "Tool": "Blast Furnace",
    "Output 1": "Gem Metal (1248)",
    "Output 2": "Lupium (3200)",
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Waterstone (10k)",
    "Catalyst": "Bor (720)",
    "Tool": "Blast Furnace",
    "Output 1": "Gem Metal (1824)",
    "Output 2": "Lupium (2048)",
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Amarantum (10k)",
    "Catalyst": "Rock Oil (420)",
    "Tool": "Furnace",
    "Output 1": null,
    "Output 2": "Electrum (144)",
    "Output 3": "Cuprum (2064)",
    "Output 4": "Bleck (960)",
    "Output 5": "Calamine (384)"
  },
  {
    "Input": "Amarantum (10k)",
    "Catalyst": "Coke (385)",
    "Tool": "Furnace",
    "Output 1": null,
    "Output 2": "Electrum (346)",
    "Output 3": "Cuprum (2400)",
    "Output 4": "Bleck (557)",
    "Output 5": "Calamine (1011)"
  },
  {
    "Input": "Blood Ore (10k)",
    "Catalyst": "Sulfur (560)",
    "Tool": "Furnace",
    "Output 1": "Pig Iron (4000)",
    "Output 2": null,
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Waterstone (10k)",
    "Catalyst": "Fuming Salt (315)",
    "Tool": "Furnace",
    "Output 1": "Gem Metal (264)",
    "Output 2": "Lupium (1800)",
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Waterstone (10k)",
    "Catalyst": "Calx Powder (715)",
    "Tool": "Furnace",
    "Output 1": "Gem Metal (180)",
    "Output 2": "Lupium (1422)",
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Waterstone (10k)",
    "Catalyst": "Sulfur (560)",
    "Tool": "Furnace",
    "Output 1": "Gem Metal (306)",
    "Output 2": "Lupium (792)",
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Galbinum (10k)",
    "Catalyst": "Fuming Salt (315)",
    "Tool": "Blast Furnace",
    "Output 1": "Pyroxene (1040)",
    "Output 2": "Lupium (384)",
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Gabore (10k)",
    "Catalyst": "Rock Oil (420)",
    "Tool": "Grinder",
    "Output 1": "Galbinum (1050)",
    "Output 2": "Blood Ore (226)",
    "Output 3": "Gabore Powder (4323)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Gabore (10k)",
    "Catalyst": "Water (1000)",
    "Tool": "Grinder",
    "Output 1": "Galbinum (998)",
    "Output 2": "Blood Ore (214)",
    "Output 3": "Gabore Powder (4095)",
    "Output 4": null,
    "Output 5": null
  },
  {
    "Input": "Granum (10k)",
    "Catalyst": "Water (1000)",
    "Tool": "Attractor",
    "Output 1": "Blood Ore (1386)",
    "Output 2": null,
    "Output 3": null,
    "Output 4": null,
    "Output 5": null
  }
]
"""

In [5]:
ExtractionRawDatum = TypedDict(
    "ExtractionRawDatum",
    {
        "Input": str,
        "Catalyst": str,
        "Tool": str,
        "Output 1": str,
        "Output 2": str,
        "Output 3": str,
        "Output 4": str,
        "Output 5": str,
    }
)
extraction_raw_data: list[ExtractionRawDatum] = json.loads(NORSCA_SHEET_CLEANED)


number_parse_re = re.compile(r"\D")


def parse_name_and_amount(input_: str) -> tuple[str, int]:
    *name_parts, quantity_raw = input_.split(" ")
    name = " ".join(name_parts)
    quantity = int(re.sub(number_parse_re, "", quantity_raw))
    if quantity_raw[-2] == "k":
        quantity *= 1000
    resource = get_resource_by_name(name)
    return resource, quantity


def parse_extraction_data() -> None:
    for d in extraction_raw_data:
        input_resource, input_amount = parse_name_and_amount(d["Input"])
        step = ProcessingStep(
            input=input_resource,
            tool=d["Tool"].strip()
        )

        if c := d.get("Catalyst"):
            catalyst_resource, catalyst_amount = parse_name_and_amount(c)
            catalyst_efficiency = ResourceEfficiency(
                catalyst_resource,
                catalyst_amount / input_amount
            )
            step.catalysts.append(catalyst_efficiency)

        for i in range(1, 6):
            if o := d.get(f"Output {i}"):
                output_resource, output_amount = parse_name_and_amount(o)
                step.outputs.append(
                    ResourceEfficiency(output_resource, output_amount / input_amount)
                )
                output_resource.add_upstream(step)

        input_resource.add_downstream(step)


parse_extraction_data()

## Parse Refining Data

In [6]:
# Assembled from mortal2.rocks extraction/refining guide
#  https://mortal2.rocks/guides/crafting/ore-extraction-calculator-mortal-online-2/
REFINING_RAW_DATA = """
[
  {
    "Input": "Pig Iron",
    "Catalyst 1": "Coke",
    "Catalyst 2": "Calx Powder",
    "Output": "Grain Steel"
  },
  {
    "Input": "Grain Steel",
    "Catalyst 1": "Coal",
    "Catalyst 2": "Saburra Powder",
    "Output": "Steel"
  },
  {
    "Input": "Cuprum",
    "Catalyst 1": "Bleck",
    "Catalyst 2": "Saburra Powder",
    "Output": "Bron"
  },
  {
    "Input": "Cuprum",
    "Catalyst 1": "Calamine",
    "Catalyst 2": "Saburra Powder",
    "Output": "Messing"
  },
  {
    "Input": "Messing",
    "Catalyst 1": "Almine",
    "Catalyst 2": "Gem Metal",
    "Output": "Tindremic Messing"
  },
  {
    "Input": "Grain Steel",
    "Catalyst 1": "Almine",
    "Catalyst 2": "Acronite",
    "Output": "Cronite"
  },
  {
    "Input": "Tungsteel",
    "Catalyst 1": "Cronite",
    "Catalyst 2": "Sanguinite",
    "Output": "Oghmium"
  },
  {
    "Input": "Grain Steel",
    "Catalyst 1": "Lupium",
    "Catalyst 2": "Granum Powder",
    "Output": "Tungsteel"
  }
]
"""

In [7]:
RefiningRawDatum = TypedDict(
    "RefiningRawDatum",
    {
        "Input": str,
        "Catalyst 1": str,
        "Catalyst 2": str,
        "Output": str,
    }
)
refining_raw_data: list[RefiningRawDatum] = json.loads(REFINING_RAW_DATA)


def parse_refining_data() -> None:
    for d in refining_raw_data:
        input_resource = get_resource_by_name(d["Input"])
        output_resource = get_resource_by_name(d["Output"])
        step = ProcessingStep(
            input=input_resource,
            tool="Refining Oven",
            catalysts=[
                ResourceEfficiency(
                    resource=get_resource_by_name(d["Catalyst 1"]),
                    factor=0.5
                ),
                ResourceEfficiency(
                    resource=get_resource_by_name(d["Catalyst 2"]),
                    factor=0.5
                )
            ],
            outputs=[
                ResourceEfficiency(
                    resource=output_resource,
                    factor=0.7
                )
            ],
        )
        input_resource.add_downstream(step)
        output_resource.add_upstream(step)


parse_refining_data()

## Define Additional Resource Attributes

In [8]:
STACK = 10000
HORSE_LOAD = STACK * 5.9

METALS = {
    get_resource_by_name(name)
    for name in [
        "Pig Iron",
        "Grain Steel",
        "Bleck",
        "Aabam",
        "Cuprum",
        "Tungsteel",
        "Steel",
        "Bron",
        "Messing",
        "Tindremic Messing",
        "Cronite",
        "Oghmium"
    ]
}


PORTABLE_TOOLS = {
    "Crusher",
    "Grinder",
    "Grizzly",
}
HOUSE_TOOLS = PORTABLE_TOOLS.union(
    {
        "Furnace",
    }
)
KEEP_TOOLS = HOUSE_TOOLS.union(
    {
        "Refining Oven",
    }
)
TOWN_TOOLS = KEEP_TOOLS.union(
    # these are in all khurite towns plus meduli and tindrem (i think?)
    {
        "Refining Oven",
        "Fabricula",
        "Kiln",
    }
)
CAPITAL_TOOLS = TOWN_TOOLS.union(
    # tindrem, MK, and hyllspeia have these
    {
        "Natorus",
        "Attractor",
        "Hearth",
    }
)


# TODO by primary point requirements?


## Resource Tree Traversal

In [9]:
def get_output_efficiency(step: ProcessingStep, resource: Resource) -> ResourceEfficiency:
    return next(o for o in step.outputs if o.resource == resource)


def get_best_upstream(resource: Resource) -> list[ProcessingStep]:
    return sorted(
        resource.upstream,
        key=lambda step: get_output_efficiency(step, resource).factor,
        reverse=True
    )


def print_best_upstream(resource: Resource) -> None:
    for step in get_best_upstream(resource):
        efficiency = get_output_efficiency(step, resource)
        print(f"{step.input} in {step.tool} with {step.catalysts} -> {resource}: {efficiency.factor}")


def traverse_upstream(resource: Resource) -> None:
    ...

In [10]:
print_best_upstream(resources["Tungsteel"])

Grain Steel in Refining Oven with [ResourceEfficiency(resource=Resource(Lupium), factor=0.5), ResourceEfficiency(resource=Resource(Granum Powder), factor=0.5)] -> Tungsteel: 0.7


In [11]:
from collections import defaultdict
from math import floor, ceil

OGHMIR = 1.03


def traverse_downstream(
        ra: ResourceAmount,
        *,
        targets: Optional[set[Resource]] = None,
        tools: Optional[set[str]] = None,
        max_depth: Optional[int] = None,
        depth: int = 0,
        external_inputs: defaultdict[Resource, int] | None = None,
        as_oghmir: bool = False
) -> None:
    if depth == max_depth:
        # print(
        #     f"{'    ' * depth}Max depth reached. "
        #     f"External inputs used: {dict(external_inputs)}"
        # )
        return

    if external_inputs is None:
        external_inputs = defaultdict(int)
        external_inputs[ra.resource] = ra.amount

    for s in ra.resource.downstream:
        if tools and s.tool not in tools:
            continue

        local_inputs: defaultdict[Resource, int] = defaultdict(int)

        for catalyst_efficiency in s.catalysts:
            catalyst_quantity = ceil(ra.amount * catalyst_efficiency.factor) * (
                OGHMIR if as_oghmir and s.tool != 'Refining Oven' else 1
            )
            local_inputs[catalyst_efficiency.resource] += catalyst_quantity

        # TODO better str methods etc, repetitive w/ the above ^
        catalysts_as_str = " and ".join(
            f"{ceil(ra.amount * e.factor * (OGHMIR if as_oghmir and s.tool != 'Refining Oven' else 1))} "
            f"{e.resource}"
            for e in s.catalysts
        )
        print(
            f"{'    ' * depth}| {ra.amount} {s.input} in {s.tool} "
            f"with {catalysts_as_str or 'no catalyst'}:"
        )
        inputs_to_here = external_inputs.copy()
        for k, v in local_inputs.items():
            inputs_to_here[k] += v

        for e in s.outputs:
            output_amount = floor(
                e.factor
                * ra.amount
                * (OGHMIR if as_oghmir else  1)
            )
            print(
                f"{'    ' * depth} -> {output_amount} {e.resource}"
            )
            # TODO:
            #  - score pipeline w/r/t target yield and surface best pipeline
            #  - prune branches that never produce the targets
            if targets and e.resource in targets:
                continue

            traverse_downstream(
                ResourceAmount(
                    e.resource,
                    output_amount
                ),
                targets=targets,
                tools=tools,
                # TODO don't love passing this state down, iterative solution
                #  is probably better than recursive anyway
                max_depth=max_depth,
                depth=depth + 1,
                external_inputs=inputs_to_here,
            )


In [12]:
traverse_downstream(
    ResourceAmount(resources["Granum"], 10000),
    # tools={"Crusher"},
    max_depth=1,
    as_oghmir=True
)

| 10000 Granum in Crusher with no catalyst:
 -> 908 Amarantum
 -> 793 Blood Ore
 -> 144 Flakestone
 -> 3028 Granum Powder
| 10000 Granum in Attractor with 737 Calx Powder:
 -> 2039 Blood Ore
| 10000 Granum in Attractor with 1030 Water:
 -> 1427 Blood Ore


In [13]:
traverse_downstream(
    ResourceAmount(resources["Blood Ore"], 10000),
    max_depth=1,
    as_oghmir=True
)

| 10000 Blood Ore in Furnace with 397 Coke:
 -> 4120 Pig Iron
| 10000 Blood Ore in Blast Furnace with 397 Coke:
 -> 5150 Pig Iron
| 10000 Blood Ore in Furnace with 577 Sulfur:
 -> 4120 Pig Iron


In [14]:
traverse_downstream(
    ResourceAmount(resources["Pig Iron"], 10000),
    max_depth=1,
    as_oghmir=True
)

| 10000 Pig Iron in Refining Oven with 5000 Coke and 5000 Calx Powder:
 -> 7210 Grain Steel


In [15]:
traverse_downstream(
    ResourceAmount(resources["Grain Steel"], 10000),
    max_depth=1,
    as_oghmir=True
)

| 10000 Grain Steel in Refining Oven with 5000 Coal and 5000 Saburra Powder:
 -> 7210 Steel
| 10000 Grain Steel in Refining Oven with 5000 Almine and 5000 Acronite:
 -> 7210 Cronite
| 10000 Grain Steel in Refining Oven with 5000 Lupium and 5000 Granum Powder:
 -> 7210 Tungsteel



## To Produce Calculations

Filter the stream to remove tools and resources (remove blast furnace, tephra, etc)

In [16]:
def combine_steps(chain: List[Tuple[str, int, str, List[Tuple[str, int]], str, float]]) -> List[Tuple[str, List[Tuple[str, int]], str, List[Tuple[str, int]], str]]:
    combined_steps = defaultdict(lambda: [defaultdict(int), [], 0])
    for output, amount, input_resource, catalysts, tool, efficiency in chain:
        key = (input_resource, tool)
        combined_steps[key][0][output] += amount
        for catalyst, c_amount in catalysts:
            found = False
            for existing_catalyst in combined_steps[key][1]:
                if existing_catalyst[0] == catalyst:
                    existing_catalyst[1] += c_amount
                    found = True
                    break
            if not found:
                combined_steps[key][1].append([catalyst, c_amount])
        combined_steps[key][2] = max(combined_steps[key][2], efficiency)

    return [(dict(outputs), sum(outputs.values()), input_resource, catalysts, tool, efficiency)
            for (input_resource, tool), (outputs, catalysts, efficiency) in combined_steps.items()]

In [17]:
def get_best_upstream_filtered(resource: Resource, removed_tools: Set[str], removed_resources: Set[str]) -> list[ProcessingStep]:
    return [
        step for step in get_best_upstream(resource)
        if step.tool not in removed_tools
        and step.input.name not in removed_resources
        and all(c.resource.name not in removed_resources for c in step.catalysts)
    ]

In [18]:
def calculate_base_materials(target_resource: str, target_amount: int, removed_tools: Set[str], removed_resources: Set[str], as_oghmir: bool = False) -> Dict[str, int]:
    base_materials = defaultdict(int)
    intermediate_products = defaultdict(int)
    stack = [(target_resource, target_amount)]
    OGHMIR = 1.03 if as_oghmir else 1.0

    while stack:
        resource, amount = stack.pop()

        if resource in ["Granum", "Calx", "Saburra", "Tephra", "Gabore"]:
            base_materials[resource] += amount
            continue

        if resource not in resources:
            raise KeyError(f"Unknown resource: {resource}")

        if intermediate_products[resource] >= amount:
            intermediate_products[resource] -= amount
            continue

        amount_to_produce = amount - intermediate_products[resource]
        intermediate_products[resource] = 0

        upstream_steps = get_best_upstream_filtered(resources[resource], removed_tools, removed_resources)

        if not upstream_steps:
            base_materials[resource] += amount_to_produce
            continue

        best_step = upstream_steps[0]
        input_efficiency = next(o for o in best_step.outputs if o.resource.name == resource)
        input_amount = ceil(amount_to_produce / (input_efficiency.factor * (OGHMIR if best_step.tool != 'Refining Oven' else 1)))

        stack.append((best_step.input.name, input_amount))

        for catalyst in best_step.catalysts:
            catalyst_amount = ceil(input_amount * catalyst.factor * (OGHMIR if best_step.tool != 'Refining Oven' else 1))
            stack.append((catalyst.resource.name, catalyst_amount))

        for output in best_step.outputs:
            if output.resource.name != resource:
                produced_amount = floor(input_amount * output.factor * (OGHMIR if best_step.tool != 'Refining Oven' else 1))
                intermediate_products[output.resource.name] += produced_amount

    return dict(base_materials)

In [19]:
import math

def get_full_production_chain(target_resource: str, target_amount: int, removed_tools: Set[str], removed_resources: Set[str], as_oghmir: bool = False) -> List[Tuple[str, int, str, List[Tuple[str, int]], str, float]]:
    chain = []
    stack = [(target_resource, target_amount)]
    intermediate_products = defaultdict(int)
    OGHMIR = 1.03 if as_oghmir else 1.0

    while stack:
        resource, amount = stack.pop()

        if resource in ["Granum", "Calx", "Saburra", "Tephra", "Gabore"] or resource not in resources:
            continue

        if intermediate_products[resource] >= amount:
            intermediate_products[resource] -= amount
            continue

        amount_to_produce = amount - intermediate_products[resource]
        intermediate_products[resource] = 0

        upstream_steps = get_best_upstream_filtered(resources[resource], removed_tools, removed_resources)

        if not upstream_steps:
            continue

        best_step = upstream_steps[0]
        input_efficiency = next(o for o in best_step.outputs if o.resource.name == resource)
        input_amount = math.ceil(amount_to_produce / (input_efficiency.factor * (OGHMIR if best_step.tool != 'Refining Oven' else 1)))

        catalysts = [(c.resource.name, math.ceil(input_amount * c.factor * (OGHMIR if best_step.tool != 'Refining Oven' else 1))) for c in best_step.catalysts]
        chain.append((resource, amount_to_produce, best_step.input.name, catalysts, best_step.tool, input_efficiency.factor * (OGHMIR if best_step.tool != 'Refining Oven' else 1)))

        stack.append((best_step.input.name, input_amount))
        stack.extend(catalysts)

        for output in best_step.outputs:
            if output.resource.name != resource:
                produced_amount = math.floor(input_amount * output.factor * (OGHMIR if best_step.tool != 'Refining Oven' else 1))
                intermediate_products[output.resource.name] += produced_amount

    return chain[::-1]

In [20]:
def print_production_steps(chain: List[Tuple[str, int, str, List[Tuple[str, int]], str, float]]):
    combined_chain = combine_steps(chain)
    for step in combined_chain:
        outputs, total_output, input_resource, catalysts, tool, efficiency = step
        catalyst_str = " and ".join([f"{c_amount} {c_name}" for c_name, c_amount in catalysts])
        input_amount = ceil(total_output / efficiency)
        output_str = ", ".join([f"{amount} {resource}" for resource, amount in outputs.items()])
        print(f"To make {output_str}:")
        print(f"  Use {input_amount} {input_resource} in a {tool}" + (f" with {catalyst_str}" if catalyst_str else ""))
        print()

In [21]:
def calculate_resources(target_resource: str, target_amount: int, oghmir: bool = False, removed_tools: Set[str] = set(), removed_resources: Set[str] = set()):
    try:
        base_mats = calculate_base_materials(target_resource, target_amount, removed_tools, removed_resources, as_oghmir=oghmir)
        chain = get_full_production_chain(target_resource, target_amount, removed_tools, removed_resources, as_oghmir=oghmir)

        print(f"To produce {target_amount} {target_resource}, you need:")
        for resource, amount in base_mats.items():
            print(f"  {amount} {resource} --> {amount/10000:.4f} Stacks")
        print("\nProduction steps:")
        print_production_steps(chain)
    except KeyError as e:
        print(f"Error: {e}")
        print("Please check the resource name and try again.")
    except ValueError as e:
        print(f"Error: {e}")
        print("This might be a base resource or there's no known production method.")

In [22]:
def remove_tools(*tools: str) -> Set[str]:
    return set(tools)

def remove_resources(*resources: str) -> Set[str]:
    return set(resources)


## Resource Calculation

In [ ]:
calculate_resources("Steel", 10000, oghmir=True, removed_tools=remove_tools("Blast Furnace", "Greater Nat"), removed_resources=remove_resources("Kimurite"))

To produce 10000 Steel, you need:
  8772 Water --> 0.8772 Stacks
  16223 Saburra --> 1.6223 Stacks
  165126 Calx --> 16.5126 Stacks
  242900 Granum --> 24.2900 Stacks

Production steps:
To make 14612 Calx Powder:
  Use 68933 Calx in a Grinder with 7101 Water

To make 49537 Blood Ore:
  Use 242900 Granum in a Attractor with 17889 Calx Powder

To make 21311 Coal:
  Use 96190 Calx in a Crusher

To make 12170 Coke:
  Use 16411 Coal in a Furnace with 906 Coal

To make 20409 Pig Iron:
  Use 49537 Blood Ore in a Furnace with 1965 Coke

To make 14286 Grain Steel:
  Use 20409 Pig Iron in a Refining Oven with 10205 Coke and 10205 Calx Powder

To make 7143 Saburra Powder:
  Use 16223 Saburra in a Grinder with 1671 Water

To make 10000 Steel:
  Use 14286 Grain Steel in a Refining Oven with 7143 Coal and 7143 Saburra Powder

